In [ ]:
spark.sql("""
    CREATE TABLE IF NOT EXISTS a6_motor_data_risk_summary (
        p_product_code STRING,
        std_v_date_issue DATE,
        std_v_date_expiry DATE,
        earn_date DATE,
        std_p_branch STRING,
        std_p_department STRING,
        std_p_code_cooperation_type STRING,
        std_p_customer_name STRING,
        p_salestaff STRING,
        coverage STRING,
        TOTAL_PREMIUM_WRITTEN DOUBLE,
        TOTAL_UWE_WRITTEN DOUBLE,
        TOTAL_CAT_WRITTEN DOUBLE,
        TOTAL_CLAIM DOUBLE,
        TOTAL_IBNR DOUBLE,
        TOTAL_CLAIM_CAP DOUBLE,
        TOTAL_CLAIM_CASE DOUBLE        
    ) USING DELTA
""")

In [17]:

spark.sql("""

WITH DATA AS
(
SELECT
    __.p_product_code,
    __.std_v_date_issue,
    __.std_v_date_expiry,
    __.earn_date,
    __.std_p_branch,
    __.std_p_branch || '-' || __.std_p_department AS std_p_department,
    __.std_p_code_cooperation_type,
    __.std_p_customer_name,
    __.p_salestaff,
    __.coverage,
    SUM(__.EARNED_PREMIUM) AS TOTAL_PREMIUM_WRITTEN,
    SUM(__.EARNED_UWE) AS TOTAL_UWE_WRITTEN,
    SUM(__.EARNED_CAT) AS TOTAL_CAT_WRITTEN
FROM a6_motor_earned_premium __
WHERE __.is_reverted = 0
GROUP BY
    __.p_product_code,
    __.std_v_date_issue,
    __.std_v_date_expiry,
    __.earn_date,
    __.std_p_branch,
    __.std_p_branch || '-' || __.std_p_department,
    __.coverage,
    __.std_p_code_cooperation_type,
    __.std_p_customer_name,
    __.p_salestaff
)
INSERT OVERWRITE TABLE a6_motor_data_risk_summary
SELECT
__.*,
A.std_c_TOTAL AS TOTAL_CLAIM,
A.std_c_IBNR  AS TOTAL_IBNR,
A.std_c_TOTAL_cap AS TOTAL_CLAIM_CAP,
A.std_c_case AS TOTAL_CLAIM_CASE
FROM DATA __
LEFT JOIN 
        (
            SELECT
            p_product_code,
            std_p_branch,
            std_p_branch || '-' || std_p_department AS std_p_department,
            coverage,
            std_p_customer_name,
            p_salestaff,
            std_c_date_accident,
            std_v_date_issue,
            std_v_date_expiry,
            std_p_code_cooperation_type,
            std_c_TOTAL,
            std_c_IBNR,
            std_c_TOTAL_cap,
            std_c_case
            FROM A9_MOTOR_DATA_CLAIM
            ) A ON A.p_product_code = __.p_product_code
                            AND    A.std_p_branch = __.std_p_branch
                            AND    A.std_p_department = __.std_p_department
                            AND    A.coverage = __.coverage 
                            AND    A.std_p_customer_name = __.std_p_customer_name
                            AND    A.p_salestaff = __.p_salestaff
                            AND    TO_DATE(A.std_c_date_accident) = TO_DATE(__.earn_date)
                            AND    TO_DATE(A.std_v_date_issue) = TO_DATE(__.std_v_date_issue)
                            AND    TO_DATE(A.std_v_date_expiry) = TO_DATE(__.std_v_date_expiry) 
                            AND    A.std_p_code_cooperation_type = __.std_p_code_cooperation_type
 """)

StatementMeta(, 760acdad-592c-47d2-81a6-920f0c22a58b, 19, Finished, Available, Finished)

DataFrame[]